# Iniciando com Word2Vec com Gensim!
  
A ideia por trás do Word2Vec é bem simples. Estamos supondo que você pode inferir o significado de uma palavra pela composição que ela mantém. Isso é análogo ao ditado popular "mostre-me seus amigos e eu direi quem você é". Então, se você tem duas palavras que têm vizinhos muito semelhantes (ou seja, o contexto de uso é aproximadamente o mesmo), então essas palavras são provavelmente muito semelhantes em significado ou pelo menos altamente relacionadas. Por exemplo, as palavras em inglês `shocked`,` appalled` e `astonished` são tipicamente usadas em um contexto similar.

A implementação do Gensim é baseada no artigo [original Word2Vec implementation by Google](https://arxiv.org/pdf/1301.3781.pdf) e foi extendido com funcionalidades novas.

### Imports e logging

Primeiro, começamos com os imports e o logging:

In [1]:
# imports necessários para configurar o logging
import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### Dataset 
Em seguida, manipularemos nosso conjunto de dados. O segredo para fazer o Word2Vec funcionar é ter muitos dados no formato de texto. Neste caso, uma boa base de dados pode ser encontrada no seguinte link: [OpinRank] (http://kavita-ganesan.com/entity-ranking-data/). Este conjunto de dados tem avaliações completas de usuários de carros e hotéis. Eu concatenei especificamente todas as avaliações de hotéis em um arquivo grande que é de cerca de 97MB comprimido e 229MB não compactado. Vamos usar o arquivo compactado. Cada linha neste arquivo representa uma revisão sobre o serviço do hotel. Você pode fazer o download do conjunto de dados do Word2Vec do OpinRank aqui.

Agora, vamos dar uma olhada mais de perto nesses dados, imprimindo a primeira linha. Você pode ver que esta é uma revisão bastante robusta.

In [2]:
data_file = "../data/reviews_data.txt.gz"

with gzip.open ('../data/reviews_data.txt.gz', 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break


b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

### Ler os arquivos dentro de listas
Agora que tivemos uma ideia do nosso conjunto de dados, podemos lê-lo em uma lista para que possamos passar isso para o modelo Word2Vec. Observe no código abaixo, que estou lendo diretamente o arquivo compactado. Também estamos fazendo um pré-processamento moderado das revisões usando `gensim.utils.simple_preprocess (line)`. Isso faz um pré-processamento básico, como tokenização, letras minúsculas, etc, e retorna uma lista de tokens (palavras). 



In [3]:
def read_input(input_file):
    """Esse método lê o arquivo de entrada no formato gzip"""
    
    logging.info("lendo arquivo {0}...esse processo pode demorar".format(input_file))
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("lido {0} revisoes".format (i))
            # feito alguns pre-processamentos e retorna uma lista de palavras
            # para cada revisão no formato texto
            yield gensim.utils.simple_preprocess (line)

# ler uma revisão tokenizada em uma lista
# cada revisão se torna um série de palavras 
# então isso se torna uma lista de listas
documents = list (read_input (data_file))
logging.info ("Terminado de ler os arquivos")    

2019-09-12 17:53:46,373 : INFO : lendo arquivo ../data/reviews_data.txt.gz...esse processo pode demorar
2019-09-12 17:53:46,375 : INFO : lido 0 revisoes
2019-09-12 17:53:48,150 : INFO : lido 10000 revisoes
2019-09-12 17:53:49,607 : INFO : lido 20000 revisoes
2019-09-12 17:53:51,244 : INFO : lido 30000 revisoes
2019-09-12 17:53:53,055 : INFO : lido 40000 revisoes
2019-09-12 17:53:54,738 : INFO : lido 50000 revisoes
2019-09-12 17:53:56,368 : INFO : lido 60000 revisoes
2019-09-12 17:53:58,196 : INFO : lido 70000 revisoes
2019-09-12 17:53:59,711 : INFO : lido 80000 revisoes
2019-09-12 17:54:01,039 : INFO : lido 90000 revisoes
2019-09-12 17:54:02,328 : INFO : lido 100000 revisoes
2019-09-12 17:54:03,610 : INFO : lido 110000 revisoes
2019-09-12 17:54:04,922 : INFO : lido 120000 revisoes
2019-09-12 17:54:06,265 : INFO : lido 130000 revisoes
2019-09-12 17:54:07,699 : INFO : lido 140000 revisoes
2019-09-12 17:54:09,041 : INFO : lido 150000 revisoes
2019-09-12 17:54:10,872 : INFO : lido 160000 r

## Treinamento do modelo Word2Vec

Treinar o modelo é bastante simples. Você apenas instancia o Word2Vec e passa os comentários que lemos na etapa anterior (os `documentos`). Então, estamos essencialmente passando uma lista de listas. Onde cada lista na lista principal contém um conjunto de tokens de uma revisão do usuário. O Word2Vec usa todos esses tokens para criar internamente um vocabulário. E por vocabulário, quero dizer um conjunto de palavras únicas.

Depois de construir o vocabulário, só precisamos chamar `train(...)` para começar a treinar o modelo Word2Vec. O treinamento no conjunto de dados [OpinRank] (http://kavita-ganesan.com/entity-ranking-data/) leva cerca de 10 minutos, portanto, seja paciente ao executar seu código neste conjunto de dados.

Nos bastidores, na verdade, estamos treinando uma rede neural simples com uma única camada oculta. Mas, na verdade, não vamos usar a rede neural após o treinamento. Em vez disso, o objetivo é aprender os pesos da camada oculta. Esses pesos são essencialmente os vetores de palavras que estamos tentando aprender.

In [4]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

2019-09-12 17:54:35,642 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2019-09-12 17:54:35,643 : INFO : collecting all words and their counts
2019-09-12 17:54:35,644 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-09-12 17:54:35,838 : INFO : PROGRESS: at sentence #10000, processed 1655714 words, keeping 25777 word types
2019-09-12 17:54:36,015 : INFO : PROGRESS: at sentence #20000, processed 3317863 words, keeping 35016 word types
2019-09-12 17:54:36,225 : INFO : PROGRESS: at sentence #30000, processed 5264072 words, keeping 47518 word types
2019-09-12 17:54:36,436 : INFO : PROGRESS: at sentence #40000, processed 7081746 words, keeping 56675 word types
2019-09-12 17:54:36,775 : INFO : PROGRESS: at sentence #50000, processed 9089491 words, keeping 63744 word types
2019-09-12 17:54:37,112 : INFO : PROGRESS: at sentence #60000, processed 11013726 words, keeping 76786 word types
2019-09-12 17:54:37,361 : INFO : PROGRESS: a

2019-09-12 17:55:11,581 : INFO : EPOCH 2 - PROGRESS: at 3.60% examples, 1113227 words/s, in_qsize 19, out_qsize 0
2019-09-12 17:55:12,586 : INFO : EPOCH 2 - PROGRESS: at 7.43% examples, 1140714 words/s, in_qsize 19, out_qsize 0
2019-09-12 17:55:13,590 : INFO : EPOCH 2 - PROGRESS: at 10.52% examples, 1116188 words/s, in_qsize 19, out_qsize 0
2019-09-12 17:55:14,592 : INFO : EPOCH 2 - PROGRESS: at 13.78% examples, 1132823 words/s, in_qsize 19, out_qsize 0
2019-09-12 17:55:15,594 : INFO : EPOCH 2 - PROGRESS: at 17.23% examples, 1148219 words/s, in_qsize 19, out_qsize 0
2019-09-12 17:55:16,594 : INFO : EPOCH 2 - PROGRESS: at 20.32% examples, 1152636 words/s, in_qsize 19, out_qsize 0
2019-09-12 17:55:17,598 : INFO : EPOCH 2 - PROGRESS: at 23.35% examples, 1138338 words/s, in_qsize 18, out_qsize 1
2019-09-12 17:55:18,602 : INFO : EPOCH 2 - PROGRESS: at 27.19% examples, 1143080 words/s, in_qsize 19, out_qsize 0
2019-09-12 17:55:19,608 : INFO : EPOCH 2 - PROGRESS: at 31.53% examples, 1147558 w

2019-09-12 17:56:07,347 : INFO : EPOCH 4 - PROGRESS: at 11.05% examples, 1181694 words/s, in_qsize 19, out_qsize 0
2019-09-12 17:56:08,351 : INFO : EPOCH 4 - PROGRESS: at 14.36% examples, 1182051 words/s, in_qsize 18, out_qsize 1
2019-09-12 17:56:09,364 : INFO : EPOCH 4 - PROGRESS: at 17.69% examples, 1180811 words/s, in_qsize 19, out_qsize 0
2019-09-12 17:56:10,366 : INFO : EPOCH 4 - PROGRESS: at 20.66% examples, 1173576 words/s, in_qsize 17, out_qsize 2
2019-09-12 17:56:11,368 : INFO : EPOCH 4 - PROGRESS: at 24.07% examples, 1176866 words/s, in_qsize 20, out_qsize 0
2019-09-12 17:56:12,386 : INFO : EPOCH 4 - PROGRESS: at 28.36% examples, 1178604 words/s, in_qsize 17, out_qsize 2
2019-09-12 17:56:13,392 : INFO : EPOCH 4 - PROGRESS: at 32.66% examples, 1179825 words/s, in_qsize 19, out_qsize 0
2019-09-12 17:56:14,408 : INFO : EPOCH 4 - PROGRESS: at 36.61% examples, 1174112 words/s, in_qsize 17, out_qsize 2
2019-09-12 17:56:15,413 : INFO : EPOCH 4 - PROGRESS: at 40.83% examples, 1176312

2019-09-12 17:56:59,556 : WARNING : Effective 'alpha' higher than previous training cycles
2019-09-12 17:56:59,557 : INFO : training model with 10 workers on 70537 vocabulary and 150 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2019-09-12 17:57:00,566 : INFO : EPOCH 1 - PROGRESS: at 2.74% examples, 858026 words/s, in_qsize 18, out_qsize 1
2019-09-12 17:57:01,580 : INFO : EPOCH 1 - PROGRESS: at 6.38% examples, 976608 words/s, in_qsize 18, out_qsize 1
2019-09-12 17:57:02,584 : INFO : EPOCH 1 - PROGRESS: at 9.79% examples, 1025590 words/s, in_qsize 19, out_qsize 0
2019-09-12 17:57:03,588 : INFO : EPOCH 1 - PROGRESS: at 12.75% examples, 1046372 words/s, in_qsize 19, out_qsize 0
2019-09-12 17:57:04,604 : INFO : EPOCH 1 - PROGRESS: at 16.29% examples, 1074137 words/s, in_qsize 17, out_qsize 2
2019-09-12 17:57:05,610 : INFO : EPOCH 1 - PROGRESS: at 19.58% examples, 1098824 words/s, in_qsize 19, out_qsize 0
2019-09-12 17:57:06,612 : INFO : EPOCH 1 - PROGRESS: at 23.02% examples,

2019-09-12 17:57:53,799 : INFO : EPOCH 3 - PROGRESS: at 3.86% examples, 1187547 words/s, in_qsize 19, out_qsize 0
2019-09-12 17:57:54,802 : INFO : EPOCH 3 - PROGRESS: at 7.58% examples, 1164307 words/s, in_qsize 20, out_qsize 0
2019-09-12 17:57:55,812 : INFO : EPOCH 3 - PROGRESS: at 10.03% examples, 1058344 words/s, in_qsize 18, out_qsize 1
2019-09-12 17:57:56,824 : INFO : EPOCH 3 - PROGRESS: at 11.69% examples, 948090 words/s, in_qsize 19, out_qsize 0
2019-09-12 17:57:57,833 : INFO : EPOCH 3 - PROGRESS: at 14.03% examples, 919320 words/s, in_qsize 17, out_qsize 2
2019-09-12 17:57:58,845 : INFO : EPOCH 3 - PROGRESS: at 16.36% examples, 897880 words/s, in_qsize 16, out_qsize 3
2019-09-12 17:57:59,858 : INFO : EPOCH 3 - PROGRESS: at 18.81% examples, 896353 words/s, in_qsize 20, out_qsize 1
2019-09-12 17:58:00,872 : INFO : EPOCH 3 - PROGRESS: at 20.72% examples, 878899 words/s, in_qsize 19, out_qsize 0
2019-09-12 17:58:01,914 : INFO : EPOCH 3 - PROGRESS: at 23.40% examples, 878379 words/s

2019-09-12 17:58:50,201 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-09-12 17:58:50,204 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-09-12 17:58:50,205 : INFO : EPOCH - 4 : training on 41519358 raw words (30347792 effective words) took 27.6s, 1098609 effective words/s
2019-09-12 17:58:51,216 : INFO : EPOCH 5 - PROGRESS: at 3.49% examples, 1077700 words/s, in_qsize 19, out_qsize 0
2019-09-12 17:58:52,222 : INFO : EPOCH 5 - PROGRESS: at 7.11% examples, 1090464 words/s, in_qsize 19, out_qsize 0
2019-09-12 17:58:53,231 : INFO : EPOCH 5 - PROGRESS: at 10.25% examples, 1085188 words/s, in_qsize 20, out_qsize 3
2019-09-12 17:58:54,232 : INFO : EPOCH 5 - PROGRESS: at 13.39% examples, 1095865 words/s, in_qsize 18, out_qsize 1
2019-09-12 17:58:55,248 : INFO : EPOCH 5 - PROGRESS: at 16.57% examples, 1095247 words/s, in_qsize 18, out_qsize 1
2019-09-12 17:58:56,253 : INFO : EPOCH 5 - PROGRESS: at 19.52% examples, 1097237 words/s, in_qsize 18, o

2019-09-12 17:59:46,619 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-09-12 17:59:46,620 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-09-12 17:59:46,623 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-09-12 17:59:46,627 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-09-12 17:59:46,628 : INFO : EPOCH - 6 : training on 41519358 raw words (30346052 effective words) took 28.8s, 1053523 effective words/s
2019-09-12 17:59:47,641 : INFO : EPOCH 7 - PROGRESS: at 3.40% examples, 1052731 words/s, in_qsize 19, out_qsize 0
2019-09-12 17:59:48,650 : INFO : EPOCH 7 - PROGRESS: at 7.08% examples, 1079876 words/s, in_qsize 19, out_qsize 1
2019-09-12 17:59:49,656 : INFO : EPOCH 7 - PROGRESS: at 10.59% examples, 1120040 words/s, in_qsize 18, out_qsize 1
2019-09-12 17:59:50,672 : INFO : EPOCH 7 - PROGRESS: at 13.31% examples, 1084047 words/s, in_qsize 18, out_qsize 1
2019-09-12 17:59:51,675 : INFO : EPOC

2019-09-12 18:00:45,824 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-09-12 18:00:45,828 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-09-12 18:00:45,829 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-09-12 18:00:45,839 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-09-12 18:00:45,844 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-09-12 18:00:45,847 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-09-12 18:00:45,848 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-09-12 18:00:45,858 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-09-12 18:00:45,859 : INFO : EPOCH - 8 : training on 41519358 raw words (30350327 effective words) took 31.0s, 978652 effective words/s
2019-09-12 18:00:46,875 : INFO : EPOCH 9 - PROGRESS: at 3.23% examples, 994075 words/s, in_qsize 18, out_qsize 1
2019-09-12 18:00:47

2019-09-12 18:01:42,931 : INFO : EPOCH 10 - PROGRESS: at 90.88% examples, 1019594 words/s, in_qsize 18, out_qsize 1
2019-09-12 18:01:43,951 : INFO : EPOCH 10 - PROGRESS: at 94.26% examples, 1017789 words/s, in_qsize 17, out_qsize 2
2019-09-12 18:01:44,951 : INFO : EPOCH 10 - PROGRESS: at 97.59% examples, 1016220 words/s, in_qsize 18, out_qsize 1
2019-09-12 18:01:45,541 : INFO : worker thread finished; awaiting finish of 9 more threads
2019-09-12 18:01:45,550 : INFO : worker thread finished; awaiting finish of 8 more threads
2019-09-12 18:01:45,568 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-09-12 18:01:45,569 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-09-12 18:01:45,570 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-09-12 18:01:45,580 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-09-12 18:01:45,581 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-09-12 18:0

(303483054, 415193580)

## Checando o output 
Este primeiro exemplo mostra um caso simples de procurar palavras semelhantes à palavra `dirty`. Tudo o que precisamos fazer aqui é chamar a função `most_similar` e fornecer a palavra` dirty` como o exemplo positivo. Isso retorna o top 10 palavras semelhantes.

In [5]:
w1 = "dirty"
model.wv.most_similar (positive=w1)

2019-09-12 18:01:45,609 : INFO : precomputing L2-norms of word weight vectors


[('filthy', 0.8653523921966553),
 ('stained', 0.7742222547531128),
 ('unclean', 0.767873227596283),
 ('grubby', 0.7551994323730469),
 ('dusty', 0.7500635385513306),
 ('smelly', 0.738541841506958),
 ('dingy', 0.7374070286750793),
 ('soiled', 0.7308399677276611),
 ('gross', 0.7202204465866089),
 ('grimy', 0.7143548727035522)]

Isso parece muito bom, certo? Vamos dar uma olhada em mais alguns. Vamos ver a similaridade de `polite`,` france` e `shocked`. 

In [6]:
# vamos ver as 6 palavras mais similares a 'polite'
w1 = ["polite"]
model.wv.most_similar (positive=w1, topn=6)

[('courteous', 0.9159219861030579),
 ('friendly', 0.8311355113983154),
 ('cordial', 0.8110564947128296),
 ('curteous', 0.7994455695152283),
 ('professional', 0.7916486263275146),
 ('attentive', 0.7834433317184448)]

In [7]:
# vamos ver as 6 palavras mais similares a 'france'
w1 = ["france"]
model.wv.most_similar(positive=w1, topn=6)

[('germany', 0.6628038883209229),
 ('england', 0.6498188972473145),
 ('canada', 0.6462444067001343),
 ('mexico', 0.6328214406967163),
 ('gaulle', 0.627234935760498),
 ('spain', 0.6269464492797852)]

In [9]:
# vamos ver as 6 palavras mais similares a 'shocked'
w1 = ["shocked"]
model.wv.most_similar (positive=w1,topn=6)

[('horrified', 0.8032431602478027),
 ('amazed', 0.7932386994361877),
 ('astonished', 0.7562350630760193),
 ('stunned', 0.7545634508132935),
 ('appalled', 0.751807689666748),
 ('dismayed', 0.7456820607185364)]

Isso é bom. Você pode até especificar vários exemplos positivos para obter coisas que estão relacionadas no contexto fornecido e usar exemplos negativos para dizer o que não deve ser considerado como relacionado. No exemplo abaixo, estamos pedindo todos os itens que se referem a *cama* (*bed* em inglês):

In [14]:
# palavras relacionadas com cama
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)

[('duvet', 0.70943284034729),
 ('blanket', 0.6876029968261719),
 ('mattress', 0.6864297389984131),
 ('matress', 0.6831876039505005),
 ('quilt', 0.676246702671051),
 ('pillowcase', 0.6585795879364014),
 ('sheets', 0.636309027671814),
 ('pillows', 0.6242099404335022),
 ('foam', 0.6231487989425659),
 ('pillowcases', 0.6110523343086243)]

### Similaridade entre duas palavras no vocabulário

Você pode até usar o modelo Word2Vec para retornar a semelhança entre duas palavras que estão presentes no vocabulário.

In [10]:
# similaridade de duas palavras diferentes
model.wv.similarity(w1="dirty", w2="smelly")

0.7385417

In [11]:
# similaridades de duas palavras idênticas
model.wv.similarity(w1="dirty", w2="dirty")

1.0

In [12]:
# similaridade de duas palavras opostas
model.wv.similarity(w1="dirty", w2="clean")

0.26686907

Debaixo dos panos, os três trechos acima calculam a semelhança de cosseno entre as duas palavras especificadas usando vetores de palavras de cada um. A partir das pontuações, faz sentido que `dirty` seja altamente similar a` smelly`, mas `dirty` é diferente de` clean`. Se você fizer uma semelhança entre duas palavras idênticas, a pontuação será 1.0, já que o intervalo da pontuação de semelhança do cosseno será sempre entre [0.0-1.0]. Você pode ler mais sobre a pontuação de semelhança de cosseno [aqui] (https://en.wikipedia.org/wiki/Cosine_similarity).

### Encontre o estranho
Você pode até usar o Word2Vec para encontrar itens estranhos com uma lista de itens.

In [18]:
# Qual dos items abaixo é o estranho da lista?
model.wv.doesnt_match(["cat","dog","france"])

/home/geanderson/anaconda3/lib/python3.7/site-packages/gensim/models/keyedvectors.py:876: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'france'

In [19]:
# Qual dos items abaixo é o estranho da lista?
model.wv.doesnt_match(["bed","pillow","duvet","car"])

'car'

## Entendendo os parâmetros que podem ser utilizados
Para treinar o modelo anteriormente, tivemos que definir alguns parâmetros. Agora, vamos tentar entender o que alguns deles significam. Para referência, este é o comando que usamos para treinar o modelo.

```
model = gensim.models.Word2Vec(documents, size=150, window=10, min_count=2, workers=10)
```

### `size`
O tamanho do vetor denso para representar cada token ou palavra. Se você tiver dados muito limitados, o tamanho deverá ser um valor muito menor. Se você tiver muitos dados, é bom experimentar vários tamanhos. Um valor de 100-150 funcionou bem para o dataset treinado.

### `window`
A distância máxima entre a palavra alvo e a palavra vizinha. Se a posição do seu vizinho for maior que a largura máxima da janela à esquerda e à direita, alguns vizinhos não serão considerados como relacionados à palavra de destino. Em teoria, uma janela menor deve fornecer termos mais relacionados. Se você tiver muitos dados, o tamanho da janela não deve importar muito, desde que seja uma janela de tamanho decente. 

### `min_count`
Contagem de frequência mínima de palavras. O modelo ignoraria as palavras que não statisfy o `min_count`. Palavras extremamente raras geralmente não são importantes, então é melhor livrar-se delas. A menos que seu conjunto de dados seja realmente pequeno, isso não afeta realmente o modelo.

### `workers`
Quantas threads deveríamos usar?

## Quando usar Word2Vec?

Existem muitos cenários de aplicações para o Word2Vec. Imagine se você precisa construir um analisador léxico. Treinar um modelo Word2Vec em grandes quantidades de comentários de usuários ajuda você a conseguir isso. Você tem um léxico não apenas para o sentimento, mas para a maioria das palavras no vocabulário.

Além dos dados de texto bruto não estruturados, você também pode usar o Word2Vec para obter dados mais estruturados. Por exemplo, se você tivesse tags para um milhão de perguntas e respostas do stackoverflow, poderia encontrar tags relacionadas a uma determinada tag e recomendar as relacionadas para exploração. Você pode fazer isso tratando cada conjunto de tags de coexistência como uma "frase" e treinar um modelo Word2Vec nesses dados. Concedido, você ainda precisa de um grande número de exemplos para fazê-lo funcionar.